### imports

In [2]:
import sys

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorboard.plugins.hparams import api as hp
from PIL import Image
from collections.abc import Mapping

import sklearn

sys.path.append('../')
import models


### Generators for loading and augmenting images

In [3]:
# train_datagen = ImageDataGenerator(rescale=1./255,
#                                      shear_range=0.1,
#                                      zoom_range=0.1,
#                                      rotation_range=5,
#                                      width_shift_range=0.1,
#                                      height_shift_range=0.1,
#                                      fill_mode='nearest')
train_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory('../data/train',
                                                    target_size=(150, 150),
                                                    batch_size=32,
                                                    class_mode='categorical')
test_generator = test_datagen.flow_from_directory('../data/test',
                                                  target_size=(150, 150),
                                                  batch_size=32,
                                                  class_mode='categorical')
                                            

Found 5144 images belonging to 3 classes.
Found 1288 images belonging to 3 classes.


### Setup for experiment

In [4]:
HP_NUM_FREEZED = hp.HParam('num_freezed', hp.Discrete([3, 5]))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'RMSprop']))

METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
  hp.hparams_config(
    hparams=[HP_NUM_FREEZED, HP_OPTIMIZER],
    metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
  )

2022-03-31 10:06:00.858108: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-03-31 10:06:00.859264: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-31 10:06:00.861461: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


### Training function

In [5]:
def train_test_model(hparams, tf_callback):
    model = models.XrayModelEfficient(hparams[HP_NUM_FREEZED])
    model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
                  optimizer=hparams[HP_OPTIMIZER],
                  metrics='accuracy')

    model.fit(train_generator,
              steps_per_epoch=100,
              epochs=1,
              callbacks=[tf_callback])
    
    _, accuracy = model.evaluate(test_generator)
    return accuracy

### Clear logs

In [6]:
%rm -rf ./logs/

### Run experiments

In [7]:
def run(run_dir, hparams, tf_callback):
  with tf.summary.create_file_writer(run_dir).as_default():
    hp.hparams(hparams)  # record the values used in this trial
    accuracy = train_test_model(hparams, tf_callback)
    tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

In [8]:
session_num = 0

log_dir = "logs/fit/" + "run-%d" % session_num
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

for num_freezed in HP_NUM_FREEZED.domain.values:
  for optimizer in HP_OPTIMIZER.domain.values:
    hparams = {
        HP_NUM_FREEZED: num_freezed,
        HP_OPTIMIZER: optimizer,
    }
    run_name = "run-%d" % session_num
    print('--- Starting trial: %s' % run_name)
    print({h.name: hparams[h] for h in hparams})
    run('logs/hparam_tuning/' + run_name, hparams, tensorboard_callback)
    session_num += 1

2022-03-31 10:06:11.136666: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2022-03-31 10:06:11.136703: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.
2022-03-31 10:06:11.137842: I tensorflow/core/profiler/lib/profiler_session.cc:172] Profiler session tear down.


--- Starting trial: run-0
{'num_freezed': 3, 'optimizer': 'RMSprop'}


2022-03-31 10:06:15.838323: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-03-31 10:06:15.854821: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2801090000 Hz


  1/100 [..............................] - ETA: 20:53 - loss: 1.0658 - accuracy: 0.4688

2022-03-31 10:06:28.930511: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2022-03-31 10:06:28.930537: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.


...

41/41 [==============================] - 93s 2s/step - loss: 0.9298 - accuracy: 0.6638
--- Starting trial: run-3
{'num_freezed': 5, 'optimizer': 'adam'}
  1/100 [..............................] - ETA: 21:21 - loss: 1.1144 - accuracy: 0.4375

2022-03-31 10:24:24.384949: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2022-03-31 10:24:24.384977: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.


  2/100 [..............................] - ETA: 3:59 - loss: 1.0902 - accuracy: 0.4219 

2022-03-31 10:24:27.113636: I tensorflow/core/profiler/lib/profiler_session.cc:71] Profiler session collecting data.
2022-03-31 10:24:27.135466: I tensorflow/core/profiler/lib/profiler_session.cc:172] Profiler session tear down.
2022-03-31 10:24:27.149809: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: logs/fit/run-0/train/plugins/profile/2022_03_31_10_24_27
2022-03-31 10:24:27.158451: I tensorflow/core/profiler/rpc/client/save_profile.cc:143] Dumped gzipped tool data for trace.json.gz to logs/fit/run-0/train/plugins/profile/2022_03_31_10_24_27/matined-asus.trace.json.gz
2022-03-31 10:24:27.203828: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: logs/fit/run-0/train/plugins/profile/2022_03_31_10_24_27
2022-03-31 10:24:27.210630: I tensorflow/core/profiler/rpc/client/save_profile.cc:143] Dumped gzipped tool data for memory_profile.json.gz to logs/fit/run-0/train/plugins/profile/2022_03_31_10_24_27/matined-asus.memory_profile

41/41 [==============================] - 97s 2s/step - loss: 0.8884 - accuracy: 0.6638


## Launch tensorboard

In [12]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
